In [4]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 

In [5]:
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean


In [6]:
def simulate_lsem_dynamic(W_all,Z: nx.DiGraph,
                 n: int,n_time:int, treatment_type: str,
                 noise_scale: float = 0.5,
                 baseline: float = 1.0) -> np.ndarray:
    """Simulate samples from LSEM.
        
        Args:
        W_all,A: weigthed DAG for instaneous relation and lagged relation
        n: number of samples in each time-stamp
        lag: degree of AR
        n_time: number of time stamp
        treatment_type: the type of the exposure {Binary, Gaussian}
        noise_scale: noise scale parameter of Gaussian distribution in the lSEM
        baseline: the baseline for the outcome
        
        Returns:
        X: [time,n, d] sample matrix
        """
    #W_array = nx.to_numpy_array(W)
    Z_array = nx.to_numpy_array(Z)
    d = Z_array.shape[0]
    #X_all = np.zeros([n_time+1,n, d])
    
    ## create the initial data
    X = np.zeros([n, d])
    W_0=W_all[0,:,:]
    ordered_vertices = list(nx.topological_sort(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph)))
    assert len(ordered_vertices) == d
    rank_A = ordered_vertices.index(0)
    for j in ordered_vertices:
        if ordered_vertices.index(j) > rank_A:
            parents = list(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph).predecessors(j))
            X[:, j] = X[:, parents].dot(W_0[parents, j]) + np.random.normal(scale=noise_scale, size=n)
        elif ordered_vertices.index(j) < rank_A:
            X[:, j] = np.random.normal(scale=noise_scale, size=n)
        else:
            if treatment_type == 'Binary':
                X[:, j] = 2 * (np.random.binomial(1, 0.5, n) - 0.5)
            elif treatment_type == 'Gaussian':
                X[:, j] = np.random.normal(scale=noise_scale, size=n)
            else:
                raise ValueError('unknown exposure type')
    X[:, d-1] += baseline
    X_all=X
    ## for follow-up time-stamps, X=XW+AZ+E
    for time in range(1,n_time+1):
        X_temp = np.matmul(X,Z_array)
        W_array=W_all[time,:,:] ## different index!
        W=nx.from_numpy_matrix(W_array,create_using=nx.DiGraph)
        for j in ordered_vertices:
            if ordered_vertices.index(j) > rank_A:
                parents = list(W.predecessors(j))
                X_temp[:, j] += X_temp[:, parents].dot(W_array[parents, j]) + np.random.normal(scale=noise_scale, size=n)
            elif ordered_vertices.index(j) < rank_A:
                X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
            else:
                if treatment_type == 'Binary':
                    X_temp[:, j] += 2 * (np.random.binomial(1, 0.5, n) - 0.5)
                elif treatment_type == 'Gaussian':
                    X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
                else:
                    raise ValueError('unknown exposure type')
        X_all=np.append(X_all,X_temp,axis=0)
        X=X_temp
    return X_all



In [7]:
import math 
def cos(x):
    return((-15+(5-x)**2)/15)

In [8]:
W_all=np.zeros([11,5, 5])
for i in range(11):
    W_all[i,0,4]=cos(i)

In [9]:
## create Z matrix
Z=np.identity(5)
Z[0,0]=0 # no correlation for treatment
Z_graph=nx.from_numpy_matrix(Z,create_using=nx.DiGraph)

## piecewise ANOCA

In [10]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import pickle
import os
import random

import torch.optim as optim
from torch.optim import lr_scheduler
import math
from utils import *

from multiprocessing import Pool
import multiprocessing
n_cores = multiprocessing.cpu_count()
from numpy.random import randn
from random import seed as rseed
from numpy.random import seed as npseed


In [11]:
# compute constraint h(A) value
def _h_A(A, m):
    expm_A = matrix_poly(A*A, m)
    h_A = torch.trace(expm_A) - m
    return h_A

prox_plus = torch.nn.Threshold(0.,0.)

def stau(w, tau):
    w1 = prox_plus(torch.abs(w)-tau)
    return torch.sign(w)*w1


def update_optimizer(optimizer, original_lr, c_A):
    '''related LR to c_A, whenever c_A gets big, reduce LR proportionally'''
    MAX_LR = 1e-2
    MIN_LR = 1e-4

    estimated_lr = original_lr / (math.log10(c_A) + 1e-10)
    if estimated_lr > MAX_LR:
        lr = MAX_LR
    elif estimated_lr < MIN_LR:
        lr = MIN_LR
    else:
        lr = estimated_lr

    # set LR
    for parame_group in optimizer.param_groups:
        parame_group['lr'] = lr

    return optimizer, lr

#===================================
# training:
#===================================

def train(epoch, lambda_A, c_A, optimizer,old_lr):
    t = time.time()
    nll_train = []
    kl_train = []
    mse_train = []
    shd_trian = []

    encoder.train()
    decoder.train()
    scheduler.step()


    # update optimizer
    optimizer, lr = update_optimizer(optimizer, old_lr, c_A)


    for batch_idx, (data, relations) in enumerate(train_loader):

#         if args.cuda:
#             data, relations = data.cuda(), relations.cuda()
        data, relations = Variable(data).double(), Variable(relations).double()

        # reshape data
        relations = relations.unsqueeze(2)

        optimizer.zero_grad()

        enc_x, logits, origin_A, adj_A_tilt_encoder, z_gap, z_positive, myA, Wa = encoder(data, rel_rec, rel_send)  # logits is of size: [num_sims, z_dims]
        edges = logits

        dec_x, output, adj_A_tilt_decoder = decoder(data, edges,d * x_dims, rel_rec, rel_send, origin_A, adj_A_tilt_encoder, Wa)

        if torch.sum(output != output):
            print('nan error\n')

        target = data
        preds = output
        variance = 0.

        # reconstruction accuracy loss
        loss_nll = nll_gaussian(preds, target, variance)

        # KL loss
        loss_kl = kl_gaussian(logits)

        # ELBO loss:
        loss = loss_kl + loss_nll

        # add A loss
        one_adj_A = origin_A # torch.mean(adj_A_tilt_decoder, dim =0)
        sparse_loss = tau_A * torch.sum(torch.abs(one_adj_A))

        # compute h(A)
        h_A = _h_A(origin_A, d)
        loss += lambda_A * h_A + 0.5 * c_A * h_A * h_A + 100. * torch.trace(origin_A*origin_A) + sparse_loss #+  0.01 * torch.sum(variance * variance)


        loss.backward()
        loss = optimizer.step()

        myA.data = stau(myA.data, tau_A*lr)


        mse_train.append(F.mse_loss(preds, target).item())
        nll_train.append(loss_nll.item())
        kl_train.append(loss_kl.item())

    #print(h_A.item())

    return np.mean(np.mean(kl_train)  + np.mean(nll_train)), np.mean(nll_train), np.mean(mse_train), origin_A, optimizer, lr



In [12]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
x_dims = 1 # The number of input dimensions: default 1.
z_dims = d # The number of latent variable dimensions: default the same as variable size.
epochs = 200 # Number of epochs to train.
batch_size = 10 # Number of samples per batch. note: should be divisible by sample size, otherwise throw an error.
n_var=5

n_times=30 #no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list=np.zeros((n_times,time_stamp,n_var,n_var))
B_list=np.zeros((n_times,d, d))
FDR_total=[]
TPR_total=[]
SHD_total=[]
time_list=[]
for replicate in range(n_times):
  seed=seed_list[replicate]
  print(seed)
  X_all=simulate_lsem_dynamic(W_all,Z_graph,30,10, 'Binary',noise_scale=0.1).reshape(330,5,1) #create data
  average_list=np.zeros((time_stamp,d, d))
  FDR_list_piece=[]
  TPR_list_piece=[]
  SHD_list_piece=[]
  base_DAG=np.zeros((5, 5))
  ####estimate at each time_stamp####
  timestart=time.time()
  for j in range(time_stamp):
  # ----------- Configurations:
      k_max_iter = int(1e2) # The max iteration number for searching parameters.
      original_lr = 3e-3  # Initial learning rate.
      encoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      decoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      temp = 0.5 # Temperature for Gumbel softmax.
      factor = True # Factor graph model.
      encoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      decoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      tau_A = 0. # Coefficient for L-1 norm of matrix B.
      lambda1 = 0. # Coefficient for DAG constraint h1(B).
      c_B = 1 # Coefficient for absolute value h1(B).
      h1_tol = 1e-8 # The tolerance of error of h1(B) to zero.
      lr_decay = 200 # After how many epochs to decay LR by a factor of gamma. 
      gamma = 1.0 # LR decay factor. 
      ######################


      X=X_all[(j*30):(j*30+30),:]


      np.random.seed(seed)
      random.seed(seed)
      torch.manual_seed(seed)
      feat_train = torch.FloatTensor(X)
      feat_valid = torch.FloatTensor(X)
      feat_test = torch.FloatTensor(X)

      # Reconstruct itself
      train_data = TensorDataset(feat_train, feat_train)
      valid_data = TensorDataset(feat_valid, feat_train)
      test_data = TensorDataset(feat_test, feat_train)

      train_loader = DataLoader(train_data, batch_size = batch_size)
      valid_loader = DataLoader(valid_data, batch_size = batch_size)
      test_loader = DataLoader(test_data, batch_size = batch_size)

      # ----------- Load modules:
      off_diag = np.ones([d, d]) - np.eye(d) # Generate off-diagonal interaction graph
      rel_rec = np.array(encode_onehot(np.where(off_diag)[1]), dtype = np.float64)
      rel_send = np.array(encode_onehot(np.where(off_diag)[0]), dtype = np.float64)
      rel_rec = torch.DoubleTensor(rel_rec)
      rel_send = torch.DoubleTensor(rel_send)
      adj_A = np.zeros((d, d)) # Add adjacency matrix

      encoder = MLPEncoder(d * x_dims, x_dims, encoder_hidden,
                              int(z_dims), adj_A,
                              batch_size = batch_size,
                              do_prob = encoder_dropout, factor = factor).double()
      decoder = MLPDecoder(d * x_dims,
                              z_dims, x_dims, encoder,
                              data_variable_size = d,
                              batch_size = batch_size,
                              n_hid=decoder_hidden,
                              do_prob=decoder_dropout).double()

      # ----------- Set up optimizer:
      optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr = original_lr)
      scheduler = lr_scheduler.StepLR(optimizer, step_size = lr_decay,
                                      gamma = gamma)

      rel_rec = Variable(rel_rec)
      rel_send = Variable(rel_send)

      # ----------- Main:
      best_ELBO_loss = np.inf
      best_NLL_loss = np.inf
      best_MSE_loss = np.inf
      h1_B_new = torch.tensor(1.)
      h2_B_new = 1
      h1_B_old = np.inf
      h2_B_old = np.inf
      lr = original_lr
    

      try:
          for step_k in range(k_max_iter):
              while c_B< 1e+20:
                  for epoch in range(epochs):
                      old_lr = lr 
                      ELBO_loss, NLL_loss, MSE_loss, origin_B, optimizer, lr = train(epoch, lambda1, c_B, optimizer, old_lr)

                      if ELBO_loss < best_ELBO_loss:
                          best_ELBO_loss = ELBO_loss

                      if NLL_loss < best_NLL_loss:
                          best_NLL_loss = NLL_loss

                      if MSE_loss < best_MSE_loss:
                          best_MSE_loss = MSE_loss

                  if ELBO_loss > 2 * best_ELBO_loss:
                      break

                  # Update parameters
                  B_new = origin_B.data.clone()
                  h1_B_new =  _h_A(B_new,d)
                  if h1_B_new.item() > 0.25 * h1_B_old:
                      c_B *= 10
                  else:
                      break

              # Update parameters    
              h1_B_old = h1_B_new.item()
              lambda1 += c_B * h1_B_new.item()

              if h1_B_new.item() <= h1_tol:
                  break

      except KeyboardInterrupt:
          print('KeyboardInterrupt')

      predB = np.matrix(origin_B.data.clone().numpy())
      print('Best ELBO Loss :', best_ELBO_loss)
      print('Best NLL Loss :', best_NLL_loss)
      print('Best MSE Loss :', best_MSE_loss)
      #calculate_effect(predB)
      print(j)
      average_list[j,:,:]=predB



  average_coef_list[replicate,:,:,:]=average_list #average coef save to matrix
  np.save("quadra_10_30_DAGGNN_lag",average_coef_list)
#   timeend=time.time()
#   time_list.append(timeend-timestart)
  #print(timeend-timestart)
  #####write at every epoch
  #df.to_csv("cos_rep10.csv")
  print(replicate)

913812


D:\Anaconda\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Best ELBO Loss : 0.0005794695438621972
Best NLL Loss : 1.9590812679450736e-06
Best MSE Loss : 7.836325071780294e-07
0
Best ELBO Loss : 0.0014366148137122383
Best NLL Loss : 2.3761866370435904e-06
Best MSE Loss : 9.50474654817436e-07
1
Best ELBO Loss : 0.0009965669656927683
Best NLL Loss : 1.189070799187383e-06
Best MSE Loss : 4.756283196749532e-07
2
Best ELBO Loss : 0.0012024249234417487
Best NLL Loss : 1.9926711341894157e-06
Best MSE Loss : 7.970684536757664e-07
3
Best ELBO Loss : 0.0018730432029432356
Best NLL Loss : 1.2933395226921396e-05
Best MSE Loss : 5.173358090768558e-06
4
Best ELBO Loss : 0.0028443011407482607
Best NLL Loss : 9.684378318574063e-06
Best MSE Loss : 3.873751327429625e-06
5
Best ELBO Loss : 0.003380170692126821
Best NLL Loss : 1.8470354041773215e-05
Best MSE Loss : 7.388141616709286e-06
6
Best ELBO Loss : 0.0035473492991382888
Best NLL Loss : 5.916372095374205e-06
Best MSE Loss : 2.3665488381496818e-06
7
Best ELBO Loss : 0.003869503730171744
Best NLL Loss : 6.4273

Best ELBO Loss : 0.0006676565396630926
Best NLL Loss : 1.2785236895266e-06
Best MSE Loss : 5.114094758106399e-07
0
Best ELBO Loss : 0.0016706603561496007
Best NLL Loss : 4.73421772833831e-06
Best MSE Loss : 1.893687091335324e-06
1
Best ELBO Loss : 0.0017054468217565127
Best NLL Loss : 3.582778345981719e-06
Best MSE Loss : 1.4331113383926874e-06
2
Best ELBO Loss : 0.0013323783938137784
Best NLL Loss : 1.1331845699666688e-06
Best MSE Loss : 4.532738279866676e-07
3
Best ELBO Loss : 0.0017955664403977582
Best NLL Loss : 1.8851550357302432e-06
Best MSE Loss : 7.540620142920973e-07
4
Best ELBO Loss : 0.0022377966435256696
Best NLL Loss : 4.284802031932866e-06
Best MSE Loss : 1.7139208127731464e-06
5
Best ELBO Loss : 0.004530156035268575
Best NLL Loss : 1.5138406074474566e-05
Best MSE Loss : 6.0553624297898265e-06
6
Best ELBO Loss : 0.0069905977526850726
Best NLL Loss : 1.9228406841653682e-05
Best MSE Loss : 7.691362736661471e-06
7
Best ELBO Loss : 0.007181199385512972
Best NLL Loss : 1.90585

Best ELBO Loss : 0.0008664561473829464
Best NLL Loss : 1.1830912748589285e-06
Best MSE Loss : 4.732365099435714e-07
0
Best ELBO Loss : 0.0015596474395025
Best NLL Loss : 3.004044508887761e-06
Best MSE Loss : 1.2016178035551045e-06
1
Best ELBO Loss : 0.0017130510332415481
Best NLL Loss : 3.306962250728766e-06
Best MSE Loss : 1.3227849002915065e-06
2
Best ELBO Loss : 0.0018138998997825999
Best NLL Loss : 3.1656020237323147e-06
Best MSE Loss : 1.266240809492926e-06
3
Best ELBO Loss : 0.0023649716499709672
Best NLL Loss : 9.959169660234947e-06
Best MSE Loss : 3.983667864093978e-06
4
Best ELBO Loss : 0.002505475089688279
Best NLL Loss : 1.4468741615052503e-05
Best MSE Loss : 5.787496646021001e-06
5
Best ELBO Loss : 0.003081308495635063
Best NLL Loss : 5.720695973485139e-06
Best MSE Loss : 2.288278389394056e-06
6
Best ELBO Loss : 0.002664413666551569
Best NLL Loss : 3.869743914737332e-06
Best MSE Loss : 1.547897565894933e-06
7
Best ELBO Loss : 0.0031934951793291537
Best NLL Loss : 1.11115445

Best ELBO Loss : 0.0007557301875587557
Best NLL Loss : 1.3921503771967917e-06
Best MSE Loss : 5.568601508787167e-07
0
Best ELBO Loss : 0.001488766066591578
Best NLL Loss : 2.5109327695811647e-06
Best MSE Loss : 1.004373107832466e-06
1
Best ELBO Loss : 0.0013816856885100184
Best NLL Loss : 1.0733924675219174e-06
Best MSE Loss : 4.29356987008767e-07
2
Best ELBO Loss : 0.0017844531519652278
Best NLL Loss : 1.8296856265207118e-06
Best MSE Loss : 7.318742506082847e-07
3
Best ELBO Loss : 0.0019479072698609632
Best NLL Loss : 5.436117273306184e-06
Best MSE Loss : 2.174446909322474e-06
4
Best ELBO Loss : 0.0027885221946829203
Best NLL Loss : 1.3377499028537296e-05
Best MSE Loss : 5.350999611414919e-06
5
Best ELBO Loss : 0.0023871201697042155
Best NLL Loss : 9.620312255483512e-06
Best MSE Loss : 3.848124902193404e-06
6
Best ELBO Loss : 0.002929237051260788
Best NLL Loss : 1.607778435208608e-05
Best MSE Loss : 6.431113740834432e-06
7
Best ELBO Loss : 0.0035272219566456296
Best NLL Loss : 7.37499

Best ELBO Loss : 0.0015360416481800087
Best NLL Loss : 4.631643804578602e-06
Best MSE Loss : 1.852657521831441e-06
0
Best ELBO Loss : 0.0028454900478387946
Best NLL Loss : 7.4071769666193425e-06
Best MSE Loss : 2.9628707866477373e-06
1
Best ELBO Loss : 0.002368789212971471
Best NLL Loss : 4.107185819846282e-06
Best MSE Loss : 1.6428743279385128e-06
2
Best ELBO Loss : 0.0036842892187553193
Best NLL Loss : 8.025838262527404e-06
Best MSE Loss : 3.2103353050109615e-06
3
Best ELBO Loss : 0.0036751240935715083
Best NLL Loss : 6.848959453373274e-06
Best MSE Loss : 2.73958378134931e-06
4
Best ELBO Loss : 0.005454305489732032
Best NLL Loss : 2.0119515884412234e-05
Best MSE Loss : 8.047806353764894e-06
5
Best ELBO Loss : 0.006125234212999026
Best NLL Loss : 2.7108074473755422e-05
Best MSE Loss : 1.0843229789502169e-05
6
Best ELBO Loss : 0.008550102535799117
Best NLL Loss : 3.893814979118238e-05
Best MSE Loss : 1.557525991647295e-05
7
Best ELBO Loss : 0.009066755513400143
Best NLL Loss : 4.450955